In [1]:
import tensorflow as tf

In [0]:
#!pip install -U tensorflow==2.0

In [3]:
tf.__version__

'1.15.0'

In [0]:
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


1. Read the dataset

In [0]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Churn.csv')


In [7]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


2. Drop the columns

In [0]:
data_df=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [10]:




data_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00



3. Distinguish the feature and target set

In [0]:

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data_df['Geography'] = le.fit_transform(data_df['Geography'])

data_df['Gender'] = le.fit_transform(data_df['Gender'])

In [0]:
Y=pd.DataFrame(data_df['Exited'])
X=data_df.drop(['Exited'],axis=1)

In [41]:


X.shape

(10000, 10)

In [15]:
Y.shape

(10000, 1)

4. Divide the data set into Train & Test 

In [0]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3)

In [0]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

5. Normalize the test and train data

In [0]:

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')



from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

In [0]:
y_train = tf.keras.utils.to_categorical(y_train)

y_test = tf.keras.utils.to_categorical(y_test)

6. Initialize & build the model

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Adding  dence layer
model.add(tf.keras.layers.Dense(20, activation='sigmoid',input_shape=(10,))) # we have 10 features thats why 10 input_shape

#Add hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid')) 

#Adding OUTPUT layer
model.add(tf.keras.layers.Dense(2, activation='softmax')) # outputs are 2

In [0]:
#Compiling the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [37]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test), 
          batch_size=32,
          epochs=10
          )

Train on 7000 samples, validate on 3000 samples
Epoch 1/10
7000/7000 [==============================] - 1s 123us/sample - loss: 0.5039 - acc: 0.7960 - val_loss: 0.5014 - val_acc: 0.7970
Epoch 2/10
7000/7000 [==============================] - 1s 122us/sample - loss: 0.5029 - acc: 0.7960 - val_loss: 0.5003 - val_acc: 0.7970
Epoch 3/10
7000/7000 [==============================] - 1s 124us/sample - loss: 0.5019 - acc: 0.7960 - val_loss: 0.4991 - val_acc: 0.7970
Epoch 4/10
7000/7000 [==============================] - 1s 124us/sample - loss: 0.5006 - acc: 0.7960 - val_loss: 0.4979 - val_acc: 0.7970
Epoch 5/10
7000/7000 [==============================] - 1s 124us/sample - loss: 0.4994 - acc: 0.7960 - val_loss: 0.4967 - val_acc: 0.7970
Epoch 6/10
7000/7000 [==============================] - 1s 124us/sample - loss: 0.4982 - acc: 0.7960 - val_loss: 0.4953 - val_acc: 0.7970
Epoch 7/10
7000/7000 [==============================] - 1s 122us/sample - loss: 0.4970 - acc: 0.7960 - val_loss: 0.4940 - va

In [0]:
# defining another model with input layer activation func. as relu and loss as binary crossentropy 

7. Optimize the model (in 5 points)



In [27]:
modela = tf.keras.models.Sequential()

modela.add(tf.keras.layers.Dense(50, input_shape=(10,),activation='relu'))

modela.add(tf.keras.layers.Dense(30, kernel_initializer='normal',activation='relu'))

modela.add(tf.keras.layers.Dense(10, kernel_initializer='normal',activation='relu'))

modela.add(tf.keras.layers.Dense(5, kernel_initializer='normal',activation='relu'))

modela.add(tf.keras.layers.Dense(2, activation='sigmoid'))

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

modela.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

modela.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7000 samples, validate on 3000 samples
Epoch 1/30
7000/7000 [==============================] - 1s 153us/sample - loss: 0.5168 - acc: 0.7951 - val_loss: 0.4409 - val_acc: 0.7970
Epoch 2/30
7000/7000 [==============================] - 1s 135us/sample - loss: 0.4306 - acc: 0.7960 - val_loss: 0.4252 - val_acc: 0.7970
Epoch 3/30
7000/7000 [==============================] - 1s 132us/sample - loss: 0.4206 - acc: 0.7960 - val_loss: 0.4217 - val_acc: 0.8262
Epoch 4/30
7000/7000 [==============================] - 1s 131us/sample - loss: 0.4119 - acc: 0.8292 - val_loss: 0.4118 - val_acc: 0.8337
Epoch 5/30
7000/7000 [==============================] - 1s 132us/sample - loss: 0.4025 - acc: 0.8352 - val_loss: 0.4018 - val_acc: 0.8345
Epoch 6/30
7000/7000 [============================

 9. Predict the results using 0.5 as a threshold

In [34]:

#Predicting the Test set results
y_pred = model.predict(X_test)
#Threshold of 50%
y_pred = (y_pred > 0.5)
y_pred

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [ True, False],
       [ True, False],
       [ True, False]])


10. Print the Accuracy score and confusion matrix

In [35]:

#Making the COnfusion Matrix
from sklearn import metrics

pd.DataFrame(metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

,0,1
0,2391,0
1,609,0


In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.797